# Using Prediction Models to Predict Esophageal Squamous Cell Carcinoma ESCC

## Background
For the past few months, I have been working with case-control data from a study by Mmbaga et al. focusing on a subtype of esophageal cancer called esophageal squamous cell carcinoma (ESCC). This data contains information for ESCC and non-ESCC patients about many potential risk factors, including behavioral, sociodemographic, and medical histories. The secondary study I am contributing to does not seek to train a predictive model and is instead focused on studying possible causal factors, but I was curious if the data could be used to train a predictive model to identify individuals who may be at most risk of developing ESCC. The secondary study has fit a multivariate logistic regression model on the data to study the associations of risk factors with ESCC, but I want to determine how well a logistic regression model can predict outcome using a train/test split. In addition, I would like to see how the other supervised models (random forest, XGBoost, deep learning) can pexgborm after tuning for hyperparameters. The main distinction I am hoping to draw between my work on the secondary study and this project is that for this project, I am less concerned with the interpretation of how the risk factors are associated with ESCC outcome, and instead focused on the potential for a prediction model to identify people who are most at risk of developing ESCC based on the data that has been gathered.

## Pre-processing
### Handling missing data
I had previously already cleaned the data using R and outputted it into a `.csv` file for use in analysis in R. There is some data for some variables missing, and in most variables, the missing data does not comprise much of the study population, but for HIV status, roughly 39% of subjects were missing data. Because the training process will require no missing data in the features, I will drop the HIV variable altogether for this particular project and will drop the subjects that are missing any data for any of the other variables, which resulted in the number of subjects being reduced from 932 to 861.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('data/cleaned_data.csv')
df = df.drop(columns='Unnamed: 0') # drop first column that contains no data
print(f'Subject count: {len(df)}')
print(df.isna().sum()/len(df)) # examine missing data

Subject count: 932
zone                0.000000
casecon             0.000000
age                 0.000000
gender              0.000000
edu                 0.151288
occupation          0.000000
iwi                 0.023605
famhis              0.017167
hiv                 0.394850
smoking             0.001073
tobacco             0.001073
secondhandsmoke     0.011803
alcohol             0.000000
localbrew           0.023605
farmwork            0.000000
pesticide           0.000000
preservation        0.000000
treatgrainnuts      0.002146
cookingsite         0.003219
wellwater           0.000000
currentfirewood     0.000000
childfiresleep      0.001073
burnedtongue        0.001073
atesoilclay         0.000000
dailyteeth          0.005365
beansmagadi         0.001073
rawgreensdaily      0.002146
fruitsdaily         0.004292
smokedfishdaily     0.002146
spicychilisdaily    0.000000
dtype: float64


In [2]:
df = df.drop(columns='hiv') # drop hiv due to high missingness
df = df.dropna() # drop rows that contain missing data
print(f'Final subject count: {len(df)}')

Final subject count: 718


### Preparation of categorical variables
In this dataset, all variables except age are categorical, so I will use OneHotEncoder to process them first.

I will split the data into 70/30 for training and testing, and I want to use the same split for training all models I fit so that they can be compared.

In [3]:
df_encode = pd.DataFrame(df[['casecon','age']])
df_encode

,casecon,age
1,Control,58
4,Case,77
5,Control,77
6,Case,32
7,Control,40
...,...,...
926,Case,56
927,Control,57
928,Case,38
929,Control,38


In [4]:
from sklearn.preprocessing import OneHotEncoder

cat_var = df.drop(columns=['casecon', 'age']).columns.tolist() # all variables except casecon (outcome) and age (continuous)

df_encode = pd.DataFrame(df[['casecon','age']]) # casecon and age untouched, initialize new data frame

for var in cat_var: # for each cat variable
    cat_col = pd.DataFrame(df[var]) # extract variable from df
    # apply onehotencoder
    transform = OneHotEncoder()
    encoded = transform.fit_transform(cat_col)
    encoded = pd.DataFrame(encoded.toarray())
    # new column names
    encoded.columns = transform.get_feature_names_out()
    # concatenate new columns to encoded df
    df_encode = pd.concat([df_encode, encoded], axis=1)

# NaNs were generated in casecon and age columns due to encoding transformation
# dropping NaN rows brings back the original number of subjects after dropping missing data
df_encode = df_encode.dropna()

In [ ]:
# casecon is outcome column
x = df_encode.drop(columns='casecon')
y = df_encode['casecon']

# split 70/30 train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=72, shuffle=True)

## 1. Logistic Regression
### Training

In [6]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg_model = logreg.fit(x_train, y_train)

### Evaluation

In [38]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

logreg_pred = logreg_model.predict(x_test)
logreg_prob = logreg_model.predict_proba(x_test)

print(f'AUC: {roc_auc_score(y_test, logreg_prob[:, 1]):.3f}') # the second column of logreg_prob gives the probability of being class 1 aka an ESCC case
print(f'Accuracy: {accuracy_score(y_test, logreg_pred):.3f}')
print(f'Confusion matrix:\n{confusion_matrix(y_test, logreg_pred)}')
print(f'Classification report:\n{classification_report(y_test, logreg_pred)}')

AUC: 0.499
Accuracy: 0.497
Confusion matrix:
[[33 54]
 [29 49]]
Classification report:
              precision    recall  f1-score   support

        Case       0.53      0.38      0.44        87
     Control       0.48      0.63      0.54        78

    accuracy                           0.50       165
   macro avg       0.50      0.50      0.49       165
weighted avg       0.51      0.50      0.49       165



## 2. Random Forest
For my random forest model, I decided to tune using a grid search method with 5-fold cross-validation. I decided to try multiples of 10 trees in the model, from 10 to 100. I also decided to try setting a maximum depth for those trees of 5, 10, 15, or 20, as well as no maximum depth at all.
### Parameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rf_tuning = RandomForestClassifier(random_state=72)

# defining a grid search for parameters
parameters = {
    'n_estimators': (np.array(range(10))+1)*10, # test number of trees in multiples of 10 from 10 to 100
    'max_depth': [None, 5, 10, 15, 20] # test maximum depths as well as no maximum depth
}

# grid search using 5-fold cross validation
grid_search = GridSearchCV(
    estimator=rf_tuning,
    param_grid=parameters,
    scoring='roc_auc', # parameters evaluated based on AUC
    cv=5 # 5-fold
)
grid_search.fit(x_train, y_train)
best_params = grid_search.best_params_ # save best parameters to a dictionary for use in training

### Training

In [ ]:
# train using the best parameters obtained
rf = RandomForestClassifier(random_state=72, n_estimators=best_params['n_estimators'], max_depth=best_params['max_depth'])
rf_model = rf.fit(x_train, y_train)

### Evaluation

In [ ]:
rf_pred = rf_model.predict(x_test)
rf_prob = rf_model.predict_proba(x_test)

print(f'AUC: {roc_auc_score(y_test, rf_prob[:, 1]):.3f}')
print(f'Accuracy: {accuracy_score(y_test, rf_pred):.3f}')
print(f'Confusion matrix:\n{confusion_matrix(y_test, rf_pred)}')
print(f'Classification report:\n{classification_report(y_test, rf_pred)}')

AUC: 0.534
Accuracy: 0.539
Confusion matrix:
[[40 47]
 [29 49]]
Classification report:
              precision    recall  f1-score   support

        Case       0.58      0.46      0.51        87
     Control       0.51      0.63      0.56        78

    accuracy                           0.54       165
   macro avg       0.55      0.54      0.54       165
weighted avg       0.55      0.54      0.54       165



## 3. XGBoost
For XGBoost, I decided to tune the same parameters as in the Random Forest with the additional parameter of learning rate. Because XGBoost requires the outcome to be coded as 0/1 instead of using the strings as categories, I made a new variable converting `y_train` and `y_test` to a numeric form for use in XGBoost.
### Parameter Tuning

In [74]:
# convert y_train and y_test for use in XGBoost
y_train_bin = y_train.map({'Case':1, 'Control':0})
y_test_bin = y_test.map({'Case':1, 'Control':0})

In [ ]:
from xgboost import XGBClassifier

xgb_tuning = XGBClassifier(random_state=72)

# defining a grid search for parameters
parameters = {
    'n_estimators': (np.array(range(10))+1)*10, # test number of trees in multiples of 10 from 10 to 100
    'max_depth': [None, 5, 10, 15, 20], # test maximum depths as well as no maximum depth
    'learning_rate': [0.01, 0.1]
}

# grid search using 5-fold cross validation
grid_search = GridSearchCV(
    estimator=xgb_tuning,
    param_grid=parameters,
    scoring='roc_auc', # parameters evaluated based on AUC
    cv=5 # 5-fold
)
grid_search.fit(x_train, y_train_bin)
best_params = grid_search.best_params_ # save best parameters to a dictionary for use in training

### Training

In [ ]:
# train using the best parameters obtained
xgb = XGBClassifier(random_state=72, n_estimators=best_params['n_estimators'], max_depth=best_params['max_depth'], learning_rate=best_params['learning_rate'])
xgb_model = xgb.fit(x_train, y_train_bin)

### Evaluation

In [ ]:
xgb_pred = xgb_model.predict(x_test)
xgb_prob = xgb_model.predict_proba(x_test)

print(f'AUC: {roc_auc_score(y_test_bin, xgb_prob[:, 1]):.3f}')
print(f'Accuracy: {accuracy_score(y_test_bin, xgb_pred):.3f}')
print(f'Confusion matrix:\n{confusion_matrix(y_test_bin, xgb_pred)}')
print(f'Classification report:\n{classification_report(y_test_bin, xgb_pred)}')

AUC: 0.487
Accuracy: 0.497
Confusion matrix:
[[58 20]
 [63 24]]
Classification report:
              precision    recall  f1-score   support

           0       0.48      0.74      0.58        78
           1       0.55      0.28      0.37        87

    accuracy                           0.50       165
   macro avg       0.51      0.51      0.47       165
weighted avg       0.51      0.50      0.47       165



## 4. Deep Learning (Neural Networks)
While my understanding of neural networks indicates to me that it would not make for the best predictive model choice in my application, I wanted to try building a model anyway just to apply what we learned in the course and see how it performs. My data is not particularly large relative to the data that neural networks are typically used for, so I will be using two layers with lower numbers of neurons (16 and 8). I'll be applying a dropout layer between the layers to help prevent overfitting to the training data. I will then evaluate using the test dataset as I did with previous models.
### Training

In [68]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Build the model
nn_model = Sequential([
    Dense(16, activation='relu', input_dim=x_train.shape[1]),  # .shape[1] gives the number of encoded variables in x_train
    Dropout(0.15),  # dropout to prevent overfitting
    Dense(8, activation='relu'), # second layer
    Dense(1, activation='sigmoid')  # output for binary outcome
])

# Compile the model
nn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

### Evaluation

In [75]:
nn_prob = nn_model.predict(x_test)
nn_pred = (nn_prob > 0.5).astype("int32").flatten()

print(f'AUC: {roc_auc_score(y_test, nn_prob):.3f}')
print(f'Accuracy: {accuracy_score(y_test_bin, nn_pred):.3f}')
print(f'Confusion matrix:\n{confusion_matrix(y_test_bin, nn_pred)}')
print(f'Classification report:\n{classification_report(y_test_bin, nn_pred)}')

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
AUC: 0.595
Accuracy: 0.473
Confusion matrix:
[[78  0]
 [87  0]]
Classification report:
              precision    recall  f1-score   support

           0       0.47      1.00      0.64        78
           1       0.00      0.00      0.00        87

    accuracy                           0.47       165
   macro avg       0.24      0.50      0.32       165
weighted avg       0.22      0.47      0.30       165



/home/hcheng25/final-project-hcheng25-1/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/hcheng25/final-project-hcheng25-1/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/hcheng25/final-project-hcheng25-1/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_p

## Conclusions

In [77]:
print('-----Logistic Regression-----')
print(f'AUC: {roc_auc_score(y_test, logreg_prob[:, 1]):.3f}') # the second column of logreg_prob gives the probability of being class 1 aka an ESCC case
print(f'Accuracy: {accuracy_score(y_test, logreg_pred):.3f}')
print(f'Confusion matrix:\n{confusion_matrix(y_test, logreg_pred)}')
print(f'Classification report:\n{classification_report(y_test, logreg_pred)}')

print('-----Random Forest-----')
print(f'AUC: {roc_auc_score(y_test, rf_prob[:, 1]):.3f}')
print(f'Accuracy: {accuracy_score(y_test, rf_pred):.3f}')
print(f'Confusion matrix:\n{confusion_matrix(y_test, rf_pred)}')
print(f'Classification report:\n{classification_report(y_test, rf_pred)}')

print('-----XGBoost-----')
print(f'AUC: {roc_auc_score(y_test_bin, xgb_prob[:, 1]):.3f}')
print(f'Accuracy: {accuracy_score(y_test_bin, xgb_pred):.3f}')
print(f'Confusion matrix:\n{confusion_matrix(y_test_bin, xgb_pred)}')
print(f'Classification report:\n{classification_report(y_test_bin, xgb_pred)}')

print('-----Neural Network-----')
print(f'AUC: {roc_auc_score(y_test, nn_prob):.3f}')
print(f'Accuracy: {accuracy_score(y_test_bin, nn_pred):.3f}')
print(f'Confusion matrix:\n{confusion_matrix(y_test_bin, nn_pred)}')
print(f'Classification report:\n{classification_report(y_test_bin, nn_pred)}')

-----Logistic Regression-----
AUC: 0.499
Accuracy: 0.497
Confusion matrix:
[[33 54]
 [29 49]]
Classification report:
              precision    recall  f1-score   support

        Case       0.53      0.38      0.44        87
     Control       0.48      0.63      0.54        78

    accuracy                           0.50       165
   macro avg       0.50      0.50      0.49       165
weighted avg       0.51      0.50      0.49       165

-----Random Forest-----
AUC: 0.534
Accuracy: 0.539
Confusion matrix:
[[40 47]
 [29 49]]
Classification report:
              precision    recall  f1-score   support

        Case       0.58      0.46      0.51        87
     Control       0.51      0.63      0.56        78

    accuracy                           0.54       165
   macro avg       0.55      0.54      0.54       165
weighted avg       0.55      0.54      0.54       165

-----XGBoost-----
AUC: 0.487
Accuracy: 0.497
Confusion matrix:
[[58 20]
 [63 24]]
Classification report:
             

/home/hcheng25/final-project-hcheng25-1/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/hcheng25/final-project-hcheng25-1/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/hcheng25/final-project-hcheng25-1/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_p

In order of AUC, the neural network at first glance appears to perform best, followed by the Random Forest model, then logistic regression, and finally XGBoost. However, even before evaluating other metrics, I noticed that the neural network did not predict any cases in my test data; it only predicted controls, and happened to get it right 47% of the time because 47% of the test data were controls. I expected neural networks to perform quite poorly as a predictor if only because my data was small; overfitting was my primary concern through the process of coding and building the model. I feel that the prediction of only controls across the entire dataset confirms my concerns that there probably isn't enough data for the neural network to learn and predict ESCC outcomes properly

While evaluating which model I would choose to use as a predictor, I compared XGBoost and Random Forest relative to the logistic regression model because it is one of the simpler models that can be fit to this data and because I had already been working with a logistic regression model with this data, albeit not in a train/test split scenario. XGBoost and the logistic regression prediction models had very similar AUC and accuracy metrics, while the Random Forest model had higher AUC and accuracy than both.

In the context of predicting which subjects are most at risk of ESCC based on the features input into the prediction model, I feel that higher recall is more important than higher precision. Higher recall means that more high-risk individuals are being identified by the model whereas higher precision means that more of the flagged individuals are truly high risk. Because it would be more important to get high risk individuals to be evaluated by a healthcare professional even if some individuals who are not high risk are also recommended to see a doctor, I would weigh the recall metric more than the precision metric. For this reason, I feel the Random Forest would be the best of these models for this application because it has a higher recall score of 0.46 for cases compared to logistic regression (0.38) and XGBoost (0.28). Even then, I would still not feel comfortable implementing this model; a recall score of 0.46 means that only 46% of high-risk individuals are being identified for referral to a doctor, which, while better than none of them seeing a doctor, is still quite poor in performance in a clinical context.

There are also sources of bias in this data that would not make the model ideal for use in the real world. The data was gathered from a case-control study that selected based on outcome and was matched based on age and gender, so there are already several likely sources of bias in the training of the models. Because ESCC in Tanzania is still relatively unstudied, it is much too early to build a predictive model of this nature to aid in public health interventions. However, I am hopeful that as more studies are performed, this kind of predictive modeling may play a useful role in lessening the burden of ESCC in the population Tanzania.

## Refences
Mmbaga EJ, Deardorff K V, Mushi B, Mgisha W, Merritt M, Hiatt RA, et al. Characteristics of esophageal cancer cases in Tanzania. J Glob Oncol. 2017;4:1–10.

Gabel J V, Chamberlain RM, Ngoma T, Mwaiselage J, Schmid KK, Kahesa C, et al. Clinical and epidemiologic variations of esophageal cancer in Tanzania. World J Gastrointest Oncol. 2016;8(3):314.

Globocan data [Internet]. Available from: https://gco.iarc.fr/today/data/factsheets/populations/834-tanzania-united-republic-of-fact-sheets.pdf
